# Recurrent Neural Network (RNN)


An **RNN** is a type of neural network made for handling **sequences** — like sentences, time-series data, or audio.  
Unlike normal networks that treat every input independently, RNNs **remember past information** using something called a **hidden state**.

---

###  How It Works
At each time step:

hₜ = activation(Wₓₕ · xₜ + Wₕₕ · hₜ₋₁ + bₕ)
yₜ = Wₕᵧ · hₜ + bᵧ

Where:
- Input → `xₜ`  
- Previous memory → `hₜ₋₁`  
- Updated memory → `hₜ`  
- Output → `yₜ`
`Wₓₕ`, `Wₕₕ`, `Wₕᵧ` → weight matrices  
`bₕ`, `bᵧ` → bias terms  
`activation()` → non-linear function such as *tanh* or *ReLU*


Basically, it reads one input at a time, updates its memory, and produces an output — kind of like how we process words in a sentence.

---

###  Quick PyTorch Example

```python
import torch
import torch.nn as nn

rnn = nn.RNN(input_size=10, hidden_size=20, batch_first=True)
x = torch.randn(5, 3, 10)  # (batch, seq_len, input_size)

output, hidden = rnn(x)
print(output.shape, hidden.shape)


In [1]:
from pathlib import Path
root=Path("data")
root.mkdir(exist_ok=True)
path=Path(root)/"100_Unique_QA_Dataset.csv"

In [2]:
import pandas as pd

df = pd.read_csv(path)

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


### Tokenization Function

The following function takes a text input, converts it to lowercase, removes certain punctuation marks (`?` and `'`), and then splits the text into individual tokens (words).


In [3]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [4]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

### lets create dictionary 

In [5]:
#vocab
vocab={'<UNK>':0}

In [6]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [7]:
df.apply(build_vocab,axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [8]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [9]:
len(vocab)

324

In [10]:
# vectorization

In [11]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [12]:
text_to_indices("What what is campusx", vocab)

[1, 1, 2, 0]

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader

In [14]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [15]:
dataset = QADataset(df, vocab)

In [16]:
for i in range(10):
    print(dataset[i])

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))
(tensor([1, 2, 3, 4, 5, 8]), tensor([9]))
(tensor([10, 11, 12, 13, 14, 15]), tensor([16]))
(tensor([ 1,  2,  3, 17, 18, 19, 20, 21, 22]), tensor([23]))
(tensor([ 1,  2,  3, 24, 25,  5, 26, 19, 27]), tensor([28]))
(tensor([10, 29,  3, 30, 31]), tensor([32]))
(tensor([ 1,  2,  3, 33, 34,  5, 35]), tensor([36]))
(tensor([ 1,  2,  3, 37, 38, 39, 40]), tensor([41]))
(tensor([42, 43, 44, 45, 46, 47, 48]), tensor([49]))
(tensor([ 1,  2,  3, 50, 51, 19,  3, 45]), tensor([52]))


In [17]:
# creating dataloder

In [18]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [19]:
for question, answer in dataloader:
  print(question, answer)

tensor([[42, 18,  2, 62, 63,  3, 64, 18]]) tensor([[65]])
tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([[246]])
tensor([[10, 11, 12, 13, 14, 15]]) tensor([[16]])
tensor([[ 1,  2,  3,  4,  5, 73]]) tensor([[74]])
tensor([[  1,   2,   3, 146, 147,  19, 148]]) tensor([[149]])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([[114]])
tensor([[  1,   2,   3, 180, 181, 182, 183]]) tensor([[184]])
tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([[49]])
tensor([[  1,   2,   3, 212,   5,  14, 213, 214]]) tensor([[215]])
tensor([[  1,   2,   3,  37,  38,  39, 161]]) tensor([[162]])
tensor([[ 42, 101,   2,   3,  17]]) tensor([[102]])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([[316]])
tensor([[ 10, 308,   3, 309, 310]]) tensor([[311]])
tensor([[ 42, 137,   2, 138,  39, 175, 269]]) tensor([[99]])
tensor([[  1,   2,   3,   4,   5, 206]]) tensor([[207]])
tensor([[ 1,  2,  3, 69,  5,  3, 70, 71]]) tensor([[72]])
tensor([[  1,   2,   3, 146,  86,  19, 192, 193]]) tensor([[194]

In [20]:
import torch.nn as nn

##  Understanding nn.Embedding

**nn.Embedding** is like a **lookup table** for discrete items (words, categories, etc.).  
Instead of huge one-hot vectors, each item gets a **dense vector** of numbers which the model can learn during training.



### Quick Theory
- **num_embeddings** -> number of unique items (vocab size).  
- **embedding_dim** -> length of vector for each item (how much info you store per item).  
- Embedding vectors are **trainable** and updated via backprop.  
- Bigger `embedding_dim` -> more capacity to capture patterns, but does **not** directly set final output size.


In [21]:
emb=nn.Embedding(300, embedding_dim=2)
print("output shape :",emb)
print("length of the vector for each item: ",20)
print(question)
print("Max index:", question.max())
print("Min index:", question.min())
print(question.shape)
embed=emb(question)
embed

output shape : Embedding(300, 2)
length of the vector for each item:  20
tensor([[  1,   2,   3,  17, 115,  83,  84]])
Max index: tensor(115)
Min index: tensor(1)
torch.Size([1, 7])


tensor([[[-0.6092, -0.9178],
         [-0.4126,  0.0110],
         [-0.7848,  0.5087],
         [ 0.6012,  0.1675],
         [-0.3275, -0.1840],
         [ 0.1951,  0.0170],
         [-0.5300, -0.3180]]], grad_fn=<EmbeddingBackward0>)

In [22]:
x=nn.Embedding(324, embedding_dim=50)
print(dataset[0][0],dataset[0][0].shape)
a=x(dataset[0][0])
a.shape,a
#x has total 6 elements so 6,50 is shape

tensor([1, 2, 3, 4, 5, 6]) torch.Size([6])


(torch.Size([6, 50]),
 tensor([[ 0.6268,  0.8933, -1.2490, -1.8061, -0.5711,  1.2475, -2.0431,  1.6316,
           1.9809,  0.2070, -0.8729, -0.3445, -1.0653,  1.0728, -1.7473,  0.5859,
           0.5347,  0.2896,  0.9558, -0.6758, -0.8886, -1.9650,  0.7773,  1.2955,
          -0.1048, -0.8204, -1.2446,  0.2566, -0.2134,  0.4578,  1.9231, -1.2731,
           1.1389, -0.5887, -0.1669, -1.4945, -2.6910,  0.1779,  0.1468, -0.4423,
           0.4099,  0.5296,  1.0011,  0.3612,  1.4854, -1.5883, -0.5590, -0.4330,
          -0.1028, -0.5035],
         [-0.5056,  0.0574, -1.0703, -1.0891, -0.5065,  0.4366, -0.5490, -0.7197,
          -1.1399,  0.0832,  1.0241, -0.5500, -0.0943,  0.3651,  0.5156,  1.2315,
           0.6025, -0.7092, -1.0356, -1.4081, -0.5784,  0.2442, -0.7939,  1.0548,
          -0.1682, -0.2497,  1.3237,  0.9805,  0.1756, -1.4635, -1.3837, -1.1347,
          -0.3536, -1.3348,  0.3395, -1.2854,  0.4777,  1.3339, -1.1026, -1.8654,
           0.2018,  0.3537,  0.4909,  1.0389, -

In [23]:
y=nn.RNN(50,64)
b=y(a)
b

(tensor([[-2.2326e-01,  5.2824e-01, -4.3203e-01,  3.0618e-01,  7.3766e-01,
          -2.4961e-01, -5.7677e-01,  6.0965e-02,  2.6584e-01,  4.7650e-01,
          -3.2580e-01, -7.2345e-01,  3.3578e-01, -7.8081e-01, -7.0934e-01,
           3.0666e-01,  2.1000e-01,  8.0719e-01, -2.8237e-01,  8.5400e-01,
           4.6043e-01,  1.9781e-01,  3.5791e-01,  3.0283e-01,  5.6862e-01,
          -5.2513e-01,  6.5580e-01,  5.1689e-01,  7.2570e-01, -5.9893e-01,
           7.4059e-01, -5.6910e-01,  4.6036e-01,  1.7694e-01,  6.0803e-01,
           8.4658e-01, -3.0039e-01, -2.0616e-01, -2.4237e-01,  6.8960e-01,
           1.1245e-01,  5.0320e-01, -3.0582e-02,  4.0742e-01,  6.0274e-01,
           5.0528e-01, -7.1713e-01,  6.7248e-01, -6.4982e-01,  4.8335e-01,
          -4.4291e-01,  3.9940e-01, -6.8556e-01, -3.5211e-01, -6.6234e-01,
          -2.6362e-01,  2.5123e-01, -8.7922e-01,  4.1288e-01, -5.5437e-02,
          -7.8069e-01, -8.1894e-03,  1.8690e-01, -1.1994e-01],
         [-6.4569e-02,  3.7943e-02,  

In [24]:
b[0].shape,b[1].shape

(torch.Size([6, 64]), torch.Size([1, 64]))

In [25]:
#b[0] is hidden state
#b[1] is output state

In [26]:
z = nn.Linear(64, 324)
z(b[1])

tensor([[-2.7027e-01, -3.2306e-02, -2.0158e-01, -5.2658e-02, -2.7331e-01,
         -1.3031e-02, -9.8036e-02,  3.0786e-01,  6.2587e-01, -6.0868e-02,
          1.3998e-01,  2.7152e-01, -2.9447e-01,  2.8547e-01, -5.2427e-02,
          3.1036e-01, -1.9277e-01,  2.9946e-01, -1.9051e-01, -8.8082e-02,
         -2.6671e-01,  3.5434e-01, -9.5855e-03, -3.7849e-01,  2.3101e-01,
         -6.1108e-02,  4.0075e-01,  3.1820e-01, -1.0222e-01,  1.6391e-01,
         -8.3096e-02, -6.2626e-02,  2.8781e-01, -2.9206e-01,  2.6283e-01,
         -9.0673e-02,  1.2023e-01, -5.9162e-01,  2.2173e-03, -1.2746e-01,
         -6.7945e-02,  4.6058e-02, -2.7778e-01, -5.5458e-03,  3.9478e-01,
          2.4499e-01,  2.7286e-01, -4.0879e-02,  1.9903e-01,  2.4198e-01,
          1.2738e-01, -1.9824e-01,  2.5242e-01, -5.1522e-01,  2.7397e-01,
         -1.3139e-01,  2.4400e-02,  6.1550e-02,  1.8925e-01, -3.5604e-02,
          2.5617e-02, -8.1841e-02,  6.0330e-02, -1.9966e-01, -6.2148e-02,
         -3.4615e-01,  3.8530e-02,  1.

In [27]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [28]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [29]:
learning_rate = 0.001
epochs = 20

In [30]:
#create model
model = SimpleRNN(len(vocab))

In [31]:
#set loss() and optimizer() 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [32]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 528.600385
Epoch: 2, Loss: 464.811750
Epoch: 3, Loss: 391.569572
Epoch: 4, Loss: 325.293901
Epoch: 5, Loss: 269.970587
Epoch: 6, Loss: 220.492341
Epoch: 7, Loss: 174.902343
Epoch: 8, Loss: 134.985900
Epoch: 9, Loss: 103.030755
Epoch: 10, Loss: 78.455251
Epoch: 11, Loss: 60.026193
Epoch: 12, Loss: 46.929373
Epoch: 13, Loss: 37.217110
Epoch: 14, Loss: 30.049087
Epoch: 15, Loss: 24.479623
Epoch: 16, Loss: 20.319222
Epoch: 17, Loss: 17.141065
Epoch: 18, Loss: 14.555485
Epoch: 19, Loss: 12.543738
Epoch: 20, Loss: 10.850525


In [33]:
def predict(model, question, threshold=0.02):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [34]:
list(vocab.keys())[7]

'paris'

In [35]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [36]:
predict(model, "What planet our solar system?")

jupiter
